In [24]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import unicodedata
import random
from gender_guesser.detector import Detector
fake = Faker('es_ES')

In [25]:
df = pd.DataFrame()

In [26]:
df['cliente_id'] = range(1, 100001)

In [27]:
df['nombre_cliente'] = [unicodedata.normalize('NFKD', fake.first_name()).encode('ascii', 'ignore').decode('utf-8') for _ in range(len(df))]

In [28]:
df['apellido_cliente'] = [unicodedata.normalize('NFKD', fake.last_name()).encode('ascii', 'ignore').decode('utf-8') for _ in range(len(df))]

In [29]:
df['dni_cliente'] = [str(fake.nif()) for _ in range(len(df))]

In [30]:
df['email_cliente'] = [f"{nombre.lower()}.{apellido.lower()}@redhorn.com" for nombre, apellido in zip(df['nombre_cliente'], df['apellido_cliente'])]

In [31]:
beta_values = np.random.beta(2, 5)
edad_minima = 15
edad_maxima = 90
df['edad_cliente'] = round(beta_values * (edad_maxima- edad_minima) + edad_minima)

In [32]:
guesser = Detector()
df['genero_cliente'] = [guesser.get_gender(nombre) for nombre in df['nombre_cliente']]


In [33]:
df['genero_cliente'] = df['genero_cliente'].replace({
    'male': 'hombre',
    'female': 'mujer',
    'unknown': 'desconocido',
    'andy': 'desconocido',
    'mostly_male': 'hombre',
    'mostly_female': 'mujer'
})

In [34]:
df.head()

,cliente_id,nombre_cliente,apellido_cliente,dni_cliente,email_cliente,edad_cliente,genero_cliente
0,1,Maria Dolores,Carpio,79943370Q,maria dolores.carpio@redhorn.com,26,desconocido
1,2,Juan Bautista,Sans,80779122V,juan bautista.sans@redhorn.com,26,desconocido
2,3,Amando,Rossello,34090364V,amando.rossello@redhorn.com,26,hombre
3,4,Paca,Espada,04259364V,paca.espada@redhorn.com,26,mujer
4,5,Edu,Aparicio,54815561Y,edu.aparicio@redhorn.com,26,desconocido


In [35]:
from google.cloud import bigquery

client = bigquery.Client()

df.to_gbq('BBDD.tabla_clientes', project_id='dataproject-2-413010', if_exists='replace')